# 데이터 사전

[독립변수]
    
{  
- 공통사항
- subject_id: 사용자 ID
- timestamp: 타임스탬프  
}
- mACSStatus   
  m_charging: 충전여부 (0, 1)  
  0: 비충전  
  1: 충전중  

- mActivity  
  m_activity: 활동 유형 (0~8)  
   0: IN_VEHICLE - 차량 내에 있음 (자동차, 버스 등 교통수단을 이용 중인 상태)  
   1: ON_BICYCLE - 자전거 타는 중 (자전거를 타고 이동 중인 상태)  
   2: ON_FOOT - 걷는 중 (보행 중인 상태로, 걷기와 달리기를 포함하는 일반적인 활동)  
   3: STILL - 정지 상태 (움직임이 없는 상태, 앉아있거나 서있는 등)  
   4: UNKNOWN - 알 수 없음 (활동을 식별할 수 없는 상태)  
   5: TILTING - 기울어짐 (기기가 기울어지거나 방향이 변하는 상태)  
   7: WALKING - 걷는 중 (보행 중인 상태, ON_FOOT보다 더 구체적인 걷기 활동)  
   8: RUNNING - 달리는 중 (달리기 활동을 하는 상태)  

- m_Ambience  
  m_ambience: 주변 소리
  - 주변 소리 레이블과 각 레이블의 확률 목록

- mBle  
  mble: 블루투스 장치
  - 블루투스 장치 주소, 디바이스 분류, rssi 목록

- mGps  
  m_gps: 위치 정보
  - (고도, 위도, 경도, 속도)

- mLight  
  m_light
  - 주변광 (lx 단위)

- mScreenStatus  
  m_screen_use: 화면 사용 여부  
   0: 비사용,   
   1: 화면 사용 중  

- mUsageStats  
  m_usage_stats: 앱 사용
  - 앱 이름과 해당 사용 시간(밀리초 단위)

- mWifi  
  m_wifi
  - 객체 기지국 ID(bssid) 및 rssi 목록

- wHr  
  heart_rate
  - 심박수 기록

- wLight   
  w_light  
  - 주변광 (lx 단위)  

- wPedo  
  burned_calories: 소모 칼로리
  - 칼로리 수
  distance: 거리
  - 거리(미터)
  speed: 속도
  - 속도(km/h)
  step: 걸음
  - 걸음 수
  step_frequency: 보속
  - 분당 걸음 속도

[종속변수]
- subject_id: 사용자 ID  
- sleep_date: 수면 기록 날짜  
- lifelog_date: 일일 활동 날짜 (sleep_date 전날)  
- Q1: 기상 직후 전반적인 수면 품질  
   0: 개인 평균 미만,  
   1: 개인 평균 이상
- Q2: 수면 직전 피로도  
   0: 높은 수준의 피로도,  
   1: 낮은 수준의 피로도
- Q3: 수면 직전 스트레스 수준  
   0: 높은 수준의 스트레스,  
   1: 낮은 수준의 스트레스
- S1: 총 수면 시간 지침 준수 여부(TST)  
   0: 비적절,  
   1: 적절할 수 있음,  
   2: 적절  
- S2: 수면 효율성 지침 준수 여부(SE)  
   0: 부적절  
   1: 적절   
- S3: 수면 시작 지연 시간(SOL, SL)  
   0: 부적절  
   1: 적절  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import glob
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import ast
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from scipy.stats import entropy
from scipy.stats import iqr
import collections

from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from itertools import combinations

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from pytorch_tabnet.tab_network import TabNetEncoder

In [3]:
seed = 810

def set_seed(sd):
    random.seed(sd)
    np.random.seed(sd)
    os.environ['PYTHONHASHSEED'] = str(sd)
    random.seed(sd)
    np.random.seed(sd)
    torch.manual_seed(sd)
    torch.cuda.manual_seed_all(sd)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(seed)

# 데이터 불러오기 및 EDA

In [4]:
data_dir = '/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/ch2025_data_items'

In [5]:
dataframe = {}
for file_name in sorted(os.listdir(data_dir)):
  if not file_name.endswith('.parquet'):
    continue
  sensor_name = file_name.replace('ch2025_', '').replace('.parquet', '')

  df = pd.read_parquet(os.path.join(data_dir, file_name))
  dataframe[sensor_name] = df
  print(f'Loaded {sensor_name}, shape = {dataframe[sensor_name].shape}')

Loaded mACStatus, shape = (939896, 3)
Loaded mActivity, shape = (961062, 3)
Loaded mAmbience, shape = (476577, 3)
Loaded mBle, shape = (21830, 3)
Loaded mGps, shape = (800611, 3)
Loaded mLight, shape = (96258, 3)
Loaded mScreenStatus, shape = (939653, 3)
Loaded mUsageStats, shape = (45197, 3)
Loaded mWifi, shape = (76336, 3)
Loaded wHr, shape = (382918, 3)
Loaded wLight, shape = (633741, 3)
Loaded wPedo, shape = (748100, 9)


In [6]:
# parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet'))

# dataframe = {}

# for file_path in parquet_files:
#     sensor_name = os.path.basename(file_path).replace('ch2025_', '').replace('.parquet', '')
#     dataframe[sensor_name] = pd.read_parquet(file_path)
#     print(f" Loaded: {sensor_name}, shape = {dataframe[sensor_name].shape}")

In [7]:
for name, df in dataframe.items():
  print(f'== {name} ==')
  print(df.head(3), '\n')
  print(df.info(),'\n')
  print('-' * 50)

== mACStatus ==
  subject_id           timestamp  m_charging
0       id01 2024-06-26 12:03:00           0
1       id01 2024-06-26 12:04:00           0
2       id01 2024-06-26 12:05:00           0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939896 entries, 0 to 939895
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   subject_id  939896 non-null  object        
 1   timestamp   939896 non-null  datetime64[ns]
 2   m_charging  939896 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 21.5+ MB
None 

--------------------------------------------------
== mActivity ==
  subject_id           timestamp  m_activity
0       id01 2024-06-26 12:03:00           4
1       id01 2024-06-26 12:04:00           0
2       id01 2024-06-26 12:05:00           0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961062 entries, 0 to 961061
Data columns (total 3 columns):
 #   Column

In [8]:
# 결측치 및 요약
for name, df in dataframe.items():
  print(f'== {name} ==')
  print(f'Missing Value', (df.isnull().mean() * 100).round(2).to_dict())
  num_cols = df.select_dtypes(include='number').columns
  if len(num_cols) > 0:
    print(f'Summary Statistic', '\n', df[num_cols].describe().loc[['mean','std','min','max']])
  print()

== mACStatus ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_charging': 0.0}
Summary Statistic 
       m_charging
mean    0.291378
std     0.454397
min     0.000000
max     1.000000

== mActivity ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_activity': 0.0}
Summary Statistic 
       m_activity
mean    2.986991
std     1.132508
min     0.000000
max     8.000000

== mAmbience ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_ambience': 0.0}

== mBle ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_ble': 0.0}

== mGps ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_gps': 0.0}

== mLight ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_light': 0.0}
Summary Statistic 
             m_light
mean     185.176751
std     1939.970572
min        0.000000
max   334306.000000

== mScreenStatus ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_screen_use': 0.0}
Summary Statistic 
       m_screen_use
mean      0.228389
std       0.41979

In [9]:
for name, df in dataframe.items():
  if name.endswith('Light'):
    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      print(col)
      q1 = df[col].quantile(0.25)
      q3 = df[col].quantile(0.75)
      iqr = q3 - q1
      lower = q1 - (1.5 * iqr)
      upper = q3 + (1.5 * iqr)
      outliers = df[(df[col] < lower) | (df[col] > upper)]
      if not outliers.empty:
        print(f'Lower: {lower}, Upper: {upper}, Outlier length: {len(outliers)}, Outlier ratio: {(len(outliers) / len(df)) * 100:.2f}%, \n')
      df.loc[df[col] > upper, col] = 100000.0
  #   if len(col) > 0:
  #     sns.boxplot(y=df[col])
  #     plt.title(f'Boxplot of {name}')
  #     plt.show()
  # print()

m_light
Lower: -235.5, Upper: 392.5, Outlier length: 11638, Outlier ratio: 12.09%, 

w_light
Lower: -198.0, Upper: 330.0, Outlier length: 71751, Outlier ratio: 11.32%, 



In [10]:
# 초 단위 제거
for name, df in dataframe.items():
  df['timestamp'] =  df['timestamp'].dt.floor('min')
  df['date'] = df['timestamp'].dt.date
  dataframe[name] = df

In [11]:
# 변수마다 데이터프레임화
for key, df in dataframe.items():
  globals()[f'{key}_df'] = df

In [12]:
# dataframe = {
#     'mACStatus': (mACStatus_df, 'timestamp'),
#     'mActivity': (mActivity_df, 'timestamp'),
#     'mAmbience': (mAmbience_df, 'timestamp'),
#     'mBle': (mBle_df, 'timestamp'),
#     'mGps': (mGps_df, 'timestamp'),
#     'mLight': (mLight_df, 'timestamp'),
#     'mScreenStatus': (mScreenStatus_df, 'timestamp'),
#     'mUsageStats': (mUsageStats_df, 'timestamp'),
#     'mWifi': (mWifi_df, 'timestamp'),
#     'wHr': (wHr_df, 'timestamp'),
#     'wLight': (wLight_df, 'timestamp'),
#     'wPedo': (wPedo_df, 'timestamp'),
# }

# 변수별 전처리

In [13]:
# def explode(df):
#   col = [col for col in df.columns if col.startswith('m_')]
#   assert len(col) == 1
#   col = col[0]

#   explode_df = df.explode(col, ignore_index=True)
#   detail_df = pd.json_normalize(explode_df[col])
#   result_df = pd.concat([explode_df.drop(columns = col), detail_df], axis=1)

#   return result_df

In [14]:
def sleep_lifelog(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['sleep_date'] = np.where(
        df['timestamp'].dt.hour < 7,
        df['timestamp'].dt.date,
        df['timestamp'].dt.date + pd.Timedelta(days=1)
    )
    df['lifelog_date'] = pd.to_datetime(df['sleep_date']) - pd.Timedelta(days=1)

    return df

In [15]:
def find_col(df):
    col = [col for col in df.columns if col.startswith('m_') or col.startswith('w_') or col.startswith('he')]
    assert len(col) == 1
    col = col[0]

    return col

In [16]:
def cut_time(hour):
    if 22 <= hour or 0 <= hour < 1:
        return 'S3'

In [17]:
time_list = ['step_list']

def view_corr(df):
    for tl in time_list:
      numeric_cols = df.select_dtypes(include='number').columns
      tl_cols = [col for col in numeric_cols if col.startswith(tl)]
      corr_df = df[tl_cols]
      print(f'{tl}_corr_list')
      print(pd.DataFrame(corr_df.corr().drop_duplicates()))
      print('-'*50, '\n')

### mACStatus

In [18]:
def preprocessing_mACStatus(dfs):
    df = sleep_lifelog(dfs)

    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

    col = find_col(df)

    results = []
    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        status = part_group[col].values  # 0/1 상태

        # 충전상태 길이
        charging_period = sum(status) / len(status)
        row[f'{part}@charging_period'] = charging_period

        # 충전 상태 평균
        # ratio_charging = status.mean()
        # row[f'{part}@charging_ratio'] = ratio_charging

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum() / len(status)
        row[f'{part}@charging_transitions'] = transitions

        # 충전 패턴 분포
        _, counts = np.unique(status, return_counts=True)
        prob = counts / counts.sum()
        ent = entropy(prob) # 1에 가까울수록 충전 상태가 자주 바뀜
        row[f'{part}@charging_entropy'] = ent

        # 연속된 1 상태 길이
        length = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                length.append(current_len)
                current_len = 0
        if current_len > 0:
            length.append(current_len)

        # 평균 충전기간
        avg_charging_period = (np.mean(length) / len(length)) if length else 0
        row[f'{part}@charging_avg_period'] = avg_charging_period

        # 최대 충전기간
        max_charging_period = (np.max(length)/ len(length)) if length else 0
        row[f'{part}@charging_max_period'] = max_charging_period

        # 불안전성
        row[f'{part}@charging_instability'] = (1 + ent) * (1 + transitions) * (1 + charging_period)

      # 결과값 데이터프레임
      results.append(row)

    return pd.DataFrame(results).fillna(0)

pre_mACStatus_df = preprocessing_mACStatus(mACStatus_df)

# def cummean_mACStatus(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mACStatus_df = cummean_mACStatus(pre_mACStatus_df)

# def final_mACStatus(pre_mACStatus_df, cummean_mACStatus_df):
#   merge_df = pre_mACStatus_df.merge(cummean_mACStatus_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mACStatus_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

#   return final_df

# final_mACStatus_df = final_mACStatus(pre_mACStatus_df, cummean_mACStatus_df)

In [19]:
pre_mACStatus_df

,subject_id,date,S3@charging_period,S3@charging_transitions,S3@charging_entropy,S3@charging_avg_period,S3@charging_max_period,S3@charging_instability
0,id01,2024-06-26,0.224138,0.011494,0.532096,39.0,39.0,1.897054
1,id01,2024-06-27,0.000000,0.000000,0.000000,0.0,0.0,1.000000
2,id01,2024-06-28,0.759777,0.016760,0.551337,34.0,67.5,2.775761
3,id01,2024-06-29,0.000000,0.000000,0.000000,0.0,0.0,1.000000
4,id01,2024-06-30,0.083333,0.011111,0.286836,15.0,15.0,1.409562
...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.791667,0.008333,0.511740,95.0,95.0,2.731105
799,id10,2024-09-23,1.000000,0.000000,0.000000,57.0,57.0,2.000000
800,id10,2024-09-24,0.055556,0.005556,0.214559,10.0,10.0,1.289157
801,id10,2024-09-25,0.135593,0.005650,0.396883,24.0,24.0,1.595252


### mActivity

In [20]:
def preprocessing_mActivity(dfs):
  df = sleep_lifelog(dfs)

  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date

  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

  col = find_col(df)

  results = []
  for (sub, date), daily_group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':sub, 'date':date}
      for part, part_group in daily_group.groupby('time_part'):
        status = part_group[col].values

        # 활동 다양성
        activity_diversity = (part_group[col].nunique() / 8)
        row[f'{part}@activity_diversity'] = activity_diversity
        # 가장 많은 활동
        # activity_frequence = part_group[col].mode()[0] if not part_group[col].mode().empty else np.nan
        # row[f'{part}@activity_frequence'] = activity_frequence

        # 활동 유형 비율
        count = part_group[col].value_counts(normalize=True)

        # 가장 많은 활동 비율
        # row[f'{part}@activity_frequence_ratio'] = count.max()

        # 활동 다양성 분포
        ent = entropy(count)
        row[f'{part}@activity_entropy'] = ent

        # 활동 바뀐 횟수
        changes = (status[1:] != status[:-1]).sum()
        row[f'{part}@activity_change'] = changes / len(status)

        # 연속된 활동 저장
        length_activity = {}
        for j in range(9):
          length = []
          current = 0
          for val in status:
            if val == j:
              current += 1
            elif current > 0:
              length.append(current)
              current = 0
          if current > 0:
            length.append(current)
          length_activity[j] = length

        # 총 활동 수
        row[f'{part}@total_activity'] = sum(k + 1 * item
                                            for k, lst in length_activity.items()
                                            if k not in [3, 4]
                                            for item in lst) / len(length_activity)

        # # 활동 유형별 비율
        dynamic_state = [0,1,2,7,8]
        static_state = [3,5]

        # dynamic_state_cnt = np.isin(status, dynamic_state).sum()
        # row[f'{part}@dynamic_state_cnt'] = dynamic_state_cnt
        dynamic_state_ratio = sum([count.get(i,0) for i in dynamic_state])
        row[f'{part}@dynamic_state_ratio'] = dynamic_state_ratio

        # static_state_cnt = np.isin(status, static_state).sum()
        # row[f'{part}@static_state_cnt'] = static_state_cnt
        static_state_ratio = sum([count.get(i,0) for i in static_state])
        row[f'{part}@static_state_ratio'] = static_state_ratio

        # 활동 불안전성
        row[f'{part}@activity_instability'] = (1 + ent) * (1 + changes) * (1 + activity_diversity)

      results.append(row)

  return pd.DataFrame(results)

pre_mActivity_df = preprocessing_mActivity(mActivity_df)

# def cummean_mActivity(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mActivity_df = cummean_mActivity(pre_mActivity_df)

# def final_mActivity(pre_mActivity_df, cummean_mActivity_df):
#   merge_df = pre_mActivity_df.merge(cummean_mActivity_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mActivity_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mActivity_df = final_mActivity(pre_mActivity_df, cummean_mActivity_df)

In [21]:
pre_mActivity_df

,subject_id,date,S3@activity_diversity,S3@activity_entropy,S3@activity_change,S3@total_activity,S3@dynamic_state_ratio,S3@static_state_ratio,S3@activity_instability
0,id01,2024-06-26,0.250,0.087100,0.005747,0.000000,0.000000,0.017241,2.717749
1,id01,2024-06-27,0.250,0.675200,0.005556,8.111111,0.405556,0.594444,4.188000
2,id01,2024-06-28,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
3,id01,2024-06-29,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
4,id01,2024-06-30,0.250,0.432226,0.005556,3.111111,0.155556,0.844444,3.580565
...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.250,0.048194,0.016667,0.888889,0.008333,0.991667,3.930729
799,id10,2024-09-23,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
800,id10,2024-09-24,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
801,id10,2024-09-25,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000


### mAmbience

In [22]:
def preprocessing_mAmbience(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': subj, 'date': date}
      for part, part_group in group.groupby('time_part'):
        all_labels_scores = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device

          sum_score = sum(float(score) for _, score in flat if isinstance(score, (int, float, str)))
          all_labels_scores.extend([
              (label, float(normalized_score))
              for label, score in flat
              if sum_score > 0
              for normalized_score in [float(score) / sum_score]
              if 0 < float(normalized_score) < 1
              ])

          if all_labels_scores:
            label_list = [label for label, _ in all_labels_scores]
            score_list = [score for _, score in all_labels_scores]
            labels, scores = zip(*all_labels_scores)
            scores = list(scores)
            max_idx = np.argmax(scores)
            min_idx = np.argmin(scores)

            # 소음 최대
            # row[f'{part}@ambience_max_label'] = labels[max_idx]

            # 소음 최소
            # row[f'{part}@ambience_min_label'] = labels[min_idx]

            range = scores[max_idx] - scores[min_idx]
            row[f'{part}@ambience_range'] = range

            # 소음 형태 고유값
            ambience_cnt = len(set(label_list)) / len(label_list)
            row[f'{part}@ambience_cnt'] = ambience_cnt

            # 소음 형태 다양성 분포
            probs_counter = Counter(label_list)
            total = sum(probs_counter.values())
            probs = [v / total for v in probs_counter.values()]
            ent = entropy(probs)
            row[f'{part}@ambience_entropy'] = ent

            # 기초 통계량
            mean = np.mean(scores) if sum(scores) > 0 else 0
            row[f'{part}@ambience_mean'] = mean
            std = np.std(scores) if sum(scores) > 0 else 0
            row[f'{part}@ambience_std'] = std
            cov = (np.std(scores) / np.mean(scores)) if sum(scores) > 0 else 0
            row[f'{part}@ambience_cov'] = cov

            # 소음 불안전성
            row[f'{part}@ambience_instability'] = (1 + ent) * (1 + ambience_cnt) * (1 + cov)

      results.append(row)

    return pd.DataFrame(results)

pre_mAmbience_df = preprocessing_mAmbience(mAmbience_df)

# def cummean_mAmbience(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mAmbience_df = cummean_mAmbience(pre_mAmbience_df)

# def final_mAmbience(pre_mAmbience_df, cummean_mAmbience_df):
#   merge_df = pre_mAmbience_df.merge(cummean_mAmbience_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mAmbience_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mAmbience_df = final_mAmbience(pre_mAmbience_df, cummean_mAmbience_df)

In [23]:
pre_mAmbience_df

,subject_id,date,S3@ambience_range,S3@ambience_cnt,S3@ambience_entropy,S3@ambience_mean,S3@ambience_std,S3@ambience_cov,S3@ambience_instability
0,id01,2024-06-26,1.000000,0.058824,2.029098,0.188235,0.377724,2.006657,9.643192
1,id01,2024-06-27,1.000000,0.011111,1.609438,0.200000,0.400000,2.000000,7.915294
2,id01,2024-06-28,1.000000,0.026374,1.694087,0.197802,0.398183,2.013036,8.331465
3,id01,2024-06-29,1.000000,0.011111,1.609438,0.200000,0.400000,2.000000,7.915294
4,id01,2024-06-30,1.000000,0.030769,1.718237,0.197802,0.395992,2.001962,8.411123
...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.967609,0.306667,4.641312,0.100000,0.169753,1.697527,19.884316
799,id10,2024-09-23,0.957657,0.273333,3.865052,0.100000,0.121972,1.219723,13.750812
800,id10,2024-09-24,0.950654,0.157778,4.053516,0.100000,0.148862,1.488616,14.560517
801,id10,2024-09-25,0.958884,0.201111,4.444687,0.100000,0.162088,1.620881,17.139707


### mBle

In [24]:
device_class_labels = {'Unknown': ['0', '7936'],
                      'Computer': ['256', '284'],
                      'Phone': ['524'],
                      'Audio_Video':['1024', '1028', '1044', '1048', '1060', '1064', '1084'],
                      'Peripheral':['1280', '1344', '1408'],
                      'Wearable':['1796'],
                      'Health':['2324']}

device_labels = {
    'Unknown': device_class_labels['Unknown'],
    'Recognizable': []
}

for k, v in device_class_labels.items():
  if k != 'Unknown':
    device_labels['Recognizable'] += v

print(device_labels)

{'Unknown': ['0', '7936'], 'Recognizable': ['256', '284', '524', '1024', '1028', '1044', '1048', '1060', '1064', '1084', '1280', '1344', '1408', '1796', '2324']}


In [25]:
def preprocessing_mBle(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):

        rssi_list = []
        address_list = []
        class_counts = {f'class_{label}_cnt': 0 for label in device_labels}

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for device in flat:

              # rssi 목록
              rssi = int(device.get('rssi'))
              rssi_list.append(rssi)

              # address 목록
              address = str(device.get('address'))
              address_list.append(address)

              # 디바이스 장치 수
              device_class = str(device.get('device_class'))
              for label, ids in device_labels.items():
                if device_class in ids:
                  class_counts[f'class_{label}_cnt'] += 1

        # rssi 고유값
        rssi_cnt = len(set(rssi_list)) / len(rssi_list)
        row[f'{part}@rssi_cnt'] = rssi_cnt

        # rssi 다양성
        rssi_counter = Counter(address_list)
        total = sum(rssi_counter.values())
        probs = [v / total for v in rssi_counter.values()]
        ent_rssi = entropy(probs)
        row[f'{part}@rssi_entropy'] = ent_rssi

        # address 고유값
        address_cnt = len(set(address_list)) / len(address_list)
        row[f'{part}@address_cnt'] = address_cnt

        # address 다양성
        probs_counter = Counter(address_list)
        total = sum(probs_counter.values())
        probs = [v / total for v in probs_counter.values()]
        ent_address = entropy(probs)
        row[f'{part}@address_entropy'] = ent_address

        # 라벨 수
        total_count = sum(class_counts.values())
        for label in device_labels:
          if total_count > 0:
            row[f'{part}@{label}_cnt'] = class_counts[f'class_{label}_cnt'] / total_count
          else:
            row[f'{part}@{label}_cnt'] = 0
          # row[f'{part}@{label}_ratio'] = (class_counts[f'class_{label}_cnt'] / total_devices if total_devices > 0 else 0)

        # 라벨 다양성
        counts = np.array(list(class_counts.values()))
        ent_label = entropy(counts)
        row[f'{part}@label_entropy'] = ent_label

        # 기초 통계
        mean = np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_mean'] = np.mean(rssi_list) if rssi_list else 0
        min = np.min(rssi_list) if rssi_list else 0
        max = np.max(rssi_list) if rssi_list else 0
        range = np.max(rssi_list) - np.min(rssi_list)
        row[f'{part}@rssi_range'] = range
        std = np.std(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_std'] = std
        cov = np.std(rssi_list) / np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_cov'] = cov

        # mble 불안전성
        row[f'{part}@mble_instability'] = ((1 + ent_rssi) * (1 + rssi_cnt) *
         (1 + ent_address) * (1 + address_cnt) *
          (1 + ent_label) * (1 + (class_counts['class_Recognizable_cnt'] if class_counts['class_Recognizable_cnt'] else 0)) *
           (1 + cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mBle_df = preprocessing_mBle(mBle_df)

# def cummean_mBle(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mBle_df = cummean_mBle(pre_mBle_df)

# def final_mBle(pre_mBle_df, cummean_mBle_df):
#   merge_df = pre_mBle_df.merge(cummean_mBle_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mBle_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'Q1' in col:
#         final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
#       elif 'Q2_Q3' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mBle_df = final_mBle(pre_mBle_df, cummean_mBle_df)

In [26]:
pre_mBle_df

,subject_id,date,S3@rssi_cnt,S3@rssi_entropy,S3@address_cnt,S3@address_entropy,S3@Unknown_cnt,S3@Recognizable_cnt,S3@label_entropy,S3@rssi_mean,S3@rssi_range,S3@rssi_std,S3@rssi_cov,S3@mble_instability
0,id01,2024-06-26,0.550000,3.034298,0.625000,3.034298,0.8750,0.1250,0.376770,-73.950000,44.0,10.968022,-0.148317,288.411053
1,id01,2024-06-27,0.800000,1.609438,1.000000,1.609438,1.0000,0.0000,0.000000,-79.600000,18.0,5.817216,-0.073081,22.721574
2,id01,2024-06-28,0.750000,2.925203,0.833333,2.925203,0.8750,0.1250,0.376770,-73.625000,31.0,9.737438,-0.132257,236.219701
3,id01,2024-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id01,2024-06-30,0.875000,2.772589,1.000000,2.772589,0.9375,0.0625,0.233792,-70.250000,30.0,8.849435,-0.125971,115.108675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,id10,2024-09-22,0.354839,3.220853,0.483871,3.220853,1.0000,0.0000,0.000000,-86.333333,42.0,10.072141,-0.116666,31.638024
722,id10,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,id10,2024-09-24,0.362637,3.245321,0.516484,3.245321,1.0000,0.0000,0.000000,-83.978022,43.0,10.392282,-0.123750,32.633750
724,id10,2024-09-25,0.440000,3.192744,0.506667,3.192744,1.0000,0.0000,0.000000,-83.586667,49.0,13.315748,-0.159305,32.063802


### mGps

In [27]:
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    R = 6371  # 지구 반지름 (km)
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [29]:
def preprocessing_mGps(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        distances = []
        altitudes_list = []
        latitudes_list = []
        longitudes_list = []
        speeds_list = []
        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for entry in flat:
            altitudes_list.append(float(entry['altitude']))
            latitudes_list.append(float(entry['latitude']))
            longitudes_list.append(float(entry['longitude']))
            speeds_list.append(float(entry['speed']))

        # 중심점
        if latitudes_list and longitudes_list:
          center = (np.mean(latitudes_list), np.mean(longitudes_list))
          distances = [haversine(center, (lat, lng)) for lat, lng in zip(latitudes_list, longitudes_list)]

          # 중심점으로부터 최대 이동거리
          radius = max(distances) if distances else np.nan
          row[f'{part}@center_from_max_distance'] = radius

          # 중심점으로부터 총 이동거리
          center_total_distance = sum(distances)
          row[f'{part}@center_total_distance'] = center_total_distance

          coords = list(zip(latitudes_list, longitudes_list))
          if len(coords) >= 2:
            total_distance = sum(haversine(coords[i], coords[i+1]) for i in range(len(coords) - 1))
          else:
            total_distance = 0.0
          row[f'{part}@total_distance'] = total_distance

          # 이동경로
          path_tortuosity = ((center_total_distance / total_distance) if total_distance > 0 else 0)
          row[f'{part}@path_tortuosity'] = path_tortuosity

        # 고도 기초 통계량
        altitude_mean = np.mean(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_mean'] = altitude_mean
        altitude_std = np.std(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_std'] = altitude_std
        altitude_max = np.max(altitudes_list) if altitudes_list else 0
        # row[f'{part}@altitude_max'] = altitude_max
        altitude_min = np.min(altitudes_list) if altitudes_list else 0
        # row[f'{part}@altitude_min'] = altitude_min
        altitude_range = altitude_max - altitude_min
        row[f'{part}@altitude_range'] = altitude_range
        altitude_cov = np.std(altitudes_list) / np.mean(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_cov'] = altitude_cov

        # 고도 다양성
        hist_alt, _ = np.histogram(altitudes_list, bins=10, density=True)
        alt_ent = entropy(hist_alt) if hist_alt.sum() > 0 else 0
        row[f'{part}@altitude_entropy'] = alt_ent

        # 속도 기초 통계량
        speed_mean = np.mean(speeds_list) if speeds_list else 0
        row[f'{part}@speed_mean'] = speed_mean
        speed_max = np.max(speeds_list) if speeds_list else 0
        row[f'{part}@speed_max'] = speed_max
        speed_std = np.std(speeds_list) if speeds_list else 0
        row[f'{part}@speed_std'] = speed_std
        speed_cov =   np.std(speeds_list) / np.mean(speeds_list) if speeds_list else 0
        row[f'{part}@speed_cov'] = speed_cov

        # 정지 속도 수
        row[f'{part}@speed_zero_count'] =  np.sum([s == 0 for s in speeds_list]) / len(speeds_list) if speeds_list else 0

        # 속도 다양성
        hist_spd, _ = np.histogram(speeds_list, bins=10, density=True)
        speed_ent = entropy(hist_spd) if hist_spd.sum() > 0 else 0
        row[f'{part}@speed_entropy'] = speed_ent

        # 위도 경도 요약
        longitude_std = np.std(longitudes_list) if longitudes_list else 0
        row[f'{part}@longitude_std'] = longitude_std
        latitude_std = np.std(latitudes_list) if latitudes_list else 0
        row[f'{part}@latitude_std'] = latitude_std

        # mgps 불안전성
        row[f'{part}@mgps_instability'] = ((1 + speed_ent) * (1+ alt_ent) * (1 + abs(1 - path_tortuosity)) *
         (1 + speed_cov) * (1 + altitude_cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mGps_df = preprocessing_mGps(mGps_df)

# def cummean_mGps(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mGps_df = cummean_mGps(pre_mGps_df)

# def final_mGps(pre_mGps_df, cummean_mGps_df):
#   merge_df = pre_mGps_df.merge(cummean_mGps_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mGps_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mGps_df = final_mGps(pre_mGps_df, cummean_mGps_df)

In [30]:
pre_mGps_df

,subject_id,date,S3@center_from_max_distance,S3@center_total_distance,S3@total_distance,S3@path_tortuosity,S3@altitude_mean,S3@altitude_std,S3@altitude_range,S3@altitude_cov,S3@altitude_entropy,S3@speed_mean,S3@speed_max,S3@speed_std,S3@speed_cov,S3@speed_zero_count,S3@speed_entropy,S3@longitude_std,S3@latitude_std,S3@mgps_instability
0,id01,2024-06-26,0.044335,3.648018,3.973493,0.918088,103.900000,1.421085e-14,0.0,1.367743e-16,0.000000,0.404632,7.9141,0.507615,1.254513,0.003561,0.470308,0.000029,0.000017,3.586352
1,id01,2024-06-27,0.010763,3.140999,1.732740,1.812736,103.281403,1.701841e+00,6.0,1.647771e-02,0.671971,0.083925,1.1406,0.177321,2.112860,0.297297,0.883867,0.000019,0.000013,18.066373
2,id01,2024-06-28,0.242207,2.684673,2.352623,1.141140,103.900000,2.842171e-14,0.0,2.735487e-16,0.000000,0.196227,21.2428,0.677029,3.450235,0.012797,0.014619,0.000052,0.000047,5.152584
3,id01,2024-06-29,0.009802,0.881232,0.565779,1.557554,114.849400,9.877983e+00,25.1,8.600814e-02,1.137708,0.044319,3.2589,0.138729,3.130256,0.809823,0.198243,0.000008,0.000005,17.895602
4,id01,2024-06-30,0.187544,3.092591,1.596223,1.937443,107.203626,5.599662e+00,15.4,5.223389e-02,0.842614,0.087653,11.2511,0.468195,5.341456,0.524809,0.077593,0.000035,0.000050,25.669613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,id10,2024-09-22,0.024220,0.743136,0.151368,4.909477,99.895367,9.647134e-02,2.2,9.657238e-04,0.017845,0.012156,1.5356,0.074575,6.134812,0.475676,0.129517,0.000008,0.000008,40.309844
755,id10,2024-09-23,0.003763,0.465724,0.603362,0.771882,107.997285,6.985347e-02,1.8,6.468076e-04,0.011306,0.146214,0.5753,0.082665,0.565369,0.000000,1.728939,0.000006,0.000004,5.309017
756,id10,2024-09-24,0.017209,2.003536,0.665677,3.009772,99.898325,6.203926e-02,2.0,6.210241e-04,0.046155,0.055573,2.5409,0.175854,3.164362,0.422249,0.349864,0.000015,0.000006,17.710775
757,id10,2024-09-25,0.013414,2.751132,0.853862,3.221988,99.899747,1.664157e-01,7.6,1.665827e-03,0.036687,0.083242,1.7653,0.202016,2.426855,0.101706,0.578604,0.000018,0.000008,18.099371


### mLight

In [31]:
def preprocessing_mLight(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (sub, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': sub, 'date': date}
      for time_label, block_group in group.groupby('time_part'):
        light_list = block_group[col].tolist()

        # mlight 기초 통계
        mean = np.mean(light_list)
        row[f'{time_label}@mlight_mean'] = mean
        std = np.std(light_list)
        row[f'{time_label}@mlight_std'] = std
        cov = np.std(light_list) / np.mean(light_list)
        row[f'{time_label}@mlight_cov'] = cov
        max = np.max(light_list)
        # row[f'{time_label}@mlight_max'] = max
        min = np.min(light_list)
        # row[f'{time_label}@mlight_min'] = min
        range = max - min
        row[f'{time_label}@mlight_range'] = range

        # might 고유값
        light_cnt = len(set(light_list)) / len(light_list)
        row[f'{time_label}@mlight_cnt'] = light_cnt

        # mlight 다양성
        hist_light, _ = np.histogram(light_list, bins=10, density=True)
        ent = entropy(hist_light) if hist_light.sum() > 0 else 0
        row[f'{time_label}@mlight_entropy'] = ent

        # mlight 불안전성
        row[f'{time_label}@mlight_instability'] = (1 + ent) * (1 + cov) * (1 + light_cnt)

      results.append(row)

    return pd.DataFrame(results)

pre_mLight_df = preprocessing_mLight(mLight_df)

# def cummean_mLight(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mLight_df = cummean_mLight(pre_mLight_df)

# def final_mLight(pre_mLight_df, cummean_mLight_df):
#   merge_df = pre_mLight_df.merge(cummean_mLight_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mLight_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mLight_df = final_mLight(pre_mLight_df, cummean_mLight_df)

In [32]:
pre_mLight_df

,subject_id,date,S3@mlight_mean,S3@mlight_std,S3@mlight_cov,S3@mlight_range,S3@mlight_cnt,S3@mlight_entropy,S3@mlight_instability
0,id01,2024-06-26,126.526316,146.825843,1.160437,373.000000,0.526316,1.440701,8.048233
1,id01,2024-06-27,45.888889,109.473431,2.385620,360.000000,0.277778,0.556647,6.734163
2,id01,2024-06-28,72.888889,106.330663,1.458805,273.000000,0.388889,1.079735,7.102308
3,id01,2024-06-29,0.111111,0.314270,2.828427,1.000000,0.111111,0.348832,5.737673
4,id01,2024-06-30,37.944444,107.154858,2.823993,345.000000,0.222222,0.348832,6.304130
...,...,...,...,...,...,...,...,...,...
797,id10,2024-09-22,32.242501,51.841878,1.607874,202.972504,0.833333,0.887694,9.025259
798,id10,2024-09-23,0.000000,0.000000,NaN,0.000000,0.166667,0.000000,NaN
799,id10,2024-09-24,42.667502,87.398458,2.048361,292.005005,0.388889,0.960892,8.302093
800,id10,2024-09-25,98.006254,94.087100,0.960011,348.975006,0.888889,1.673142,9.896623


### mScreenStatus

In [33]:
def preprocessing_mScreenStatus(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []
    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        status = part_group[col].values

        # 총 화면 켜진 기간
        # on_screen_period = sum(status)
        # row[f'{part}on_screen_period'] = on_screen_period

        # 화면 켜진상태 비율
        on_screen_ratio = status.mean()
        row[f'{part}@on_screen_ratio'] = on_screen_ratio

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum() / len(status)
        row[f'{part}@on_screen_transitions'] = transitions

        # 사용 패턴 분포
        _, counts = np.unique(status, return_counts=True)
        prob = counts / counts.sum()
        ent = entropy(prob) # 1에 가까울수록 사용 상태가 자주 바뀜
        row[f'{part}@on_screen_entropy'] = ent

        # 연속된 1 상태 길이
        length = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                length.append(current_len)
                current_len = 0
        # 배열이 1로 끝날 때 마지막 구간 길이
        if current_len > 0:
            length.append(current_len)

        # 화면 켜진 기간 기초통계량
        avg_on_screen = np.mean(length) if length else 0
        row[f'{part}@avg_on_screen'] = avg_on_screen
        std_on_screen = np.std(length) if length else 0
        row[f'{part}@std_on_screen'] = std_on_screen
        cov_on_screen = (np.std(length) / np.mean(length)) if length else 0
        row[f'{part}@cov_on_screed'] = cov_on_screen
        max_on_screen = np.max(length) if length else 0
        # row[f'{part}@max_on_screen'] = max_on_screen
        min_on_screen = np.min(length) if length else 0
        # row[f'{part}@min_on_screen'] = min_on_screen
        range_on_screen = max_on_screen - min_on_screen
        row[f'{part}@range_on_screen'] = range_on_screen

        # 화면 불안전성
        row[f'{part}@mscreen_instability'] = (1 + ent) * (1 + transitions) * (1 + on_screen_ratio) * (1+ cov_on_screen)

      results.append(row)

    return pd.DataFrame(results)

pre_mScreenStatus_df = preprocessing_mScreenStatus(mScreenStatus_df)

# def cummean_mScreenStatus(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mScreenStatus_df = cummean_mScreenStatus(pre_mScreenStatus_df)

# def final_mScreenStatus(pre_mScreenStatus_df, cummean_mScreenStatus_df):
#   merge_df = pre_mScreenStatus_df.merge(cummean_mScreenStatus_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mScreenStatus_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mScreenStatus_df = final_mScreenStatus(pre_mScreenStatus_df, cummean_mScreenStatus_df)

In [34]:
pre_mScreenStatus_df

,subject_id,date,S3@on_screen_ratio,S3@on_screen_transitions,S3@on_screen_entropy,S3@avg_on_screen,S3@std_on_screen,S3@cov_on_screed,S3@range_on_screen,S3@mscreen_instability
0,id01,2024-06-26,0.384146,0.018293,0.666058,31.500000,28.500000,0.904762,57.0,4.472860
1,id01,2024-06-27,0.377778,0.005556,0.662966,68.000000,0.000000,0.000000,0.0,2.303926
2,id01,2024-06-28,0.188889,0.011111,0.484608,34.000000,0.000000,0.000000,0.0,1.784645
3,id01,2024-06-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
4,id01,2024-06-30,0.194444,0.016667,0.492604,17.500000,2.500000,0.142857,5.0,2.071481
...,...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.945455,0.036364,0.211688,34.666667,35.705586,1.029969,79.0,4.959219
799,id10,2024-09-23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
800,id10,2024-09-24,0.755556,0.044444,0.556149,27.200000,49.913525,1.835056,126.0,8.089334
801,id10,2024-09-25,0.626667,0.093333,0.660706,11.750000,10.744184,0.914399,37.0,5.654267


### mUsageStats

In [35]:
def preprocessing_mUsageStats(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': subj, 'date': date}
      for part, part_group in group.groupby('time_part'):
        all_labels_scores = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          total_time_sum = sum(item.get('total_time', 0) for item in flat if isinstance(item, dict))
          for item in flat:
            label = item['app_name']
            score = float(item['total_time'])
            if total_time_sum > 0:
              normalized_score = score / total_time_sum
              all_labels_scores.append((label, normalized_score))

          if all_labels_scores:
            label_list = [label for label, _ in all_labels_scores]
            score_list = [score for _, score in all_labels_scores]
            labels, scores = zip(*all_labels_scores)
            scores = list(scores)
            max_idx = np.argmax(scores)
            min_idx = np.argmin(scores)

            # 사용 최대 라벨
            # row[f'{part}@usagestats_max_label'] = labels[max_idx]

            # 사용 최소 라벨
            # row[f'{part}@usagestats_min_label'] = labels[min_idx]

            # 사용 범위
            range = scores[max_idx] - scores[min_idx]
            row[f'{part}@usagestats_range'] = range

            # 사용 고유값
            usagestats_cnt = len(set(label_list)) / len(label_list)
            row[f'{part}@usagestats_cnt'] = usagestats_cnt

            # 사용 형태 다양성 분포
            probs_counter = Counter(label_list)
            total = sum(probs_counter.values())
            probs = [v / total for v in probs_counter.values()]
            ent = entropy(probs)
            row[f'{part}@usagestats_entropy'] = ent

            # 기초 통계량
            mean = np.mean(scores) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_mean'] = mean
            std = np.std(scores) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_std'] = std
            cov = (np.std(scores) / np.mean(scores)) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_cov'] = cov

            # 사용 불안전성
            row[f'{part}@usagestats_instability'] = (1 + ent) * (1 + usagestats_cnt) * (1 + cov)

      results.append(row)

    return pd.DataFrame(results)

pre_mUsageStats_df = preprocessing_mUsageStats(mUsageStats_df)

# def cummean_mUsageStats(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mUsageStats_df = cummean_mUsageStats(pre_mUsageStats_df)

# def final_mUsageStats(pre_mUsageStats_df, cummean_mUsageStats_df):
#   merge_df = pre_mUsageStats_df.merge(cummean_mUsageStats_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mUsageStats_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mUsageStats_df = final_mUsageStats(pre_mUsageStats_df, cummean_mUsageStats_df)

In [36]:
pre_mUsageStats_df

,subject_id,date,S3@usagestats_range,S3@usagestats_cnt,S3@usagestats_entropy,S3@usagestats_mean,S3@usagestats_std,S3@usagestats_cov,S3@usagestats_instability
0,id01,2024-06-26,0.999779,0.500000,2.137793,0.250000,0.257646,1.030585,9.557333
1,id01,2024-06-27,0.980280,0.535714,2.582995,0.214286,0.283203,1.321612,12.774572
2,id01,2024-06-28,0.999397,0.476190,2.133211,0.333333,0.373301,1.119902,9.805003
3,id01,2024-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id01,2024-06-30,0.990752,0.500000,2.154783,0.200000,0.241667,1.208333,10.450217
...,...,...,...,...,...,...,...,...,...
785,id10,2024-09-22,0.995665,0.227273,1.453508,0.454545,0.417112,0.917647,5.774272
786,id10,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
787,id10,2024-09-24,0.998890,0.307692,2.097527,0.282051,0.315991,1.120331,8.588638
788,id10,2024-09-25,0.999304,0.245283,1.989429,0.283019,0.293624,1.037471,7.584860


### mwifi

In [37]:
def preprocessing_mwifi(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):

        rssi_list = []
        bssid_list = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for device in flat:

              # rssi 목록
              rssi = int(device.get('rssi'))
              rssi_list.append(rssi)

              # bssid 목록
              bssid = str(device.get('bssid'))
              bssid_list.append(bssid)

        # rssi 고유값
        rssi_cnt = len(set(rssi_list)) / len(rssi_list)
        row[f'{part}@rssi_cnt'] = rssi_cnt

        # rssi 다양성
        rssi_counter = Counter(rssi_list)
        total = sum(rssi_counter.values())
        probs = [v / total for v in rssi_counter.values()]
        ent_rssi = entropy(probs)
        row[f'{part}@rssi_entropy'] = ent_rssi

        # bssid 고유값
        bssid_cnt = len(set(bssid_list)) / len(bssid_list)
        row[f'{part}@bssid_cnt'] = bssid_cnt

        # bssid 다양성
        bssid_counter = Counter(bssid_list)
        total = sum(bssid_counter.values())
        probs = [v / total for v in bssid_counter.values()]
        ent_bssid = entropy(probs)
        row[f'{part}@bssid_entropy'] = ent_bssid

        # 알수없는 rssi 비율
        weak_rssi = [r for r in rssi_list if r < -100 or r > - 25]
        rssi_Uncommon_cnt = len(weak_rssi)
        rssi_Uncommon_ratio =  rssi_Uncommon_cnt / len(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_Uncommon_ratio'] = rssi_Uncommon_ratio

        # 기초 통계
        mean = np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_mean'] = np.mean(rssi_list) if rssi_list else 0
        min = np.min(rssi_list) if rssi_list else 0
        max = np.max(rssi_list) if rssi_list else 0
        range = np.max(rssi_list) - np.min(rssi_list)
        row[f'{part}@rssi_range'] = range
        std = np.std(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_std'] = std
        cov = np.std(rssi_list) / np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_cov'] = cov

        # mble 불안전성
        row[f'{part}@wifi_instability'] = ((1 + ent_rssi) * (1 + rssi_cnt) *
         (1 + ent_bssid) * (1 + bssid_cnt) *
           (1 + cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mWifi_df = preprocessing_mwifi(mWifi_df)

# def cummean_mWifi(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mWifi_df = cummean_mWifi(pre_mWifi_df)

# def final_mWifi(pre_mWifi_df, cummean_mWifi_df):
#   merge_df = pre_mWifi_df.merge(cummean_mWifi_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mWifi_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mWifi_df = final_mWifi(pre_mWifi_df, cummean_mWifi_df)

In [38]:
pre_mWifi_df

,subject_id,date,S3@rssi_cnt,S3@rssi_entropy,S3@bssid_cnt,S3@bssid_entropy,S3@rssi_Uncommon_ratio,S3@rssi_mean,S3@rssi_range,S3@rssi_std,S3@rssi_cov,S3@wifi_instability
0,id01,2024-06-26,0.263514,3.386794,0.121622,2.570658,0.000000,-61.912162,62.0,21.747920,-0.351271,14.400758
1,id01,2024-06-27,0.273504,3.163825,0.136752,2.538057,0.000000,-61.726496,56.0,20.013510,-0.324229,14.411960
2,id01,2024-06-28,0.296000,3.212343,0.120000,2.365578,0.000000,-57.184000,59.0,20.883442,-0.365197,13.063065
3,id01,2024-06-29,0.465116,2.803946,0.279070,2.228883,0.000000,-63.627907,53.0,19.186991,-0.301550,16.076385
4,id01,2024-06-30,0.411765,3.310701,0.152941,2.344219,0.011765,-59.682353,64.0,19.776523,-0.331363,15.689260
...,...,...,...,...,...,...,...,...,...,...,...,...
780,id10,2024-09-22,0.320896,3.499297,0.231343,3.084748,0.000000,-70.947761,50.0,12.417679,-0.175026,24.660277
781,id10,2024-09-23,0.278571,3.223898,0.278571,3.498532,0.000000,-65.728571,51.0,16.058306,-0.244312,23.473428
782,id10,2024-09-24,0.181452,3.396601,0.157258,3.330308,0.000000,-72.701613,49.0,11.240035,-0.154605,22.006029
783,id10,2024-09-25,0.166667,3.324686,0.240310,3.673100,0.000000,-72.689922,54.0,10.454997,-0.143830,25.037832


### wHr

In [39]:
def preprocessing_wHr(dfs):
  df = sleep_lifelog(dfs)
  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
  col = find_col(df)

  results = []

  for (sub, date), group in df.groupby(['subject_id', 'date']):
    row = {'subject_id':sub, 'date': date}

    for part, part_group in group.groupby('time_part'):
      hr_list = []
      for entry in part_group[col]:
        flat = ast.literal_eval(entry) if isinstance(entry, str) else entry
        hr_list.extend([int(hr) for hr in flat if hr is not None])

      # 심장박동 기초 통계량
      mean = np.mean(hr_list) if hr_list else 0
      row[f'{part}@hr_mean'] = mean
      std = np.std(hr_list) if hr_list else 0
      row[f'{part}@hr_std'] = std
      max = np.max(hr_list) if hr_list else 0
      # row[f'{part}@hr_max'] = max
      min = np.min(hr_list) if hr_list else 0
      # row[f'{part}@hr_min'] = min
      range = max - min
      row[f'{part}@hr_range'] = range
      cov = (np.std(hr_list) / np.mean(hr_list)) if hr_list else 0
      row[f'{part}@hr_cov'] = cov

      # 비정상적인 박동
      hr_over = len([hr for hr in hr_list if hr >= 100]) / len(hr_list)
      # row[f'{part}@hr_100_over_cnt'] = hr_over
      hr_under = len([hr for hr in hr_list if hr <= 60]) / len(hr_list)
      # row[f'{part}@hr_60_under_cnt'] = hr_under
      hr_abnormal = hr_over + hr_under
      row[f'{part}@hr_abnormal_cnt'] = hr_abnormal

      # 심장박동 다양성 분포
      hist, _ = np.histogram(hr_list, bins=12, range=(40, 160), density=True)
      ent = entropy(hist)
      row[f'{part}@hr_entropy'] = ent

      # 심장박동 불안전성
      row[f'{part}@hr_instability'] = (1 + ent) * (1 + cov) * (1 + hr_abnormal)

    results.append(row)

  return pd.DataFrame(results)

pre_wHr_df = preprocessing_wHr(wHr_df)

# def cummean_wHr(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_wHr_df = cummean_wHr(pre_wHr_df)

# def final_wHr(pre_wHr_df, cummean_wHr_df):
#   merge_df = pre_wHr_df.merge(cummean_wHr_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_wHr_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)
#   return final_df

# final_wHr_df = final_wHr(pre_wHr_df, cummean_wHr_df)

In [40]:
pre_wHr_df

,subject_id,date,S3@hr_mean,S3@hr_std,S3@hr_range,S3@hr_cov,S3@hr_abnormal_cnt,S3@hr_entropy,S3@hr_instability
0,id01,2024-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id01,2024-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id01,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id01,2024-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id01,2024-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
681,id10,2024-09-22,90.967132,6.451679,62.0,0.070923,0.060333,0.932441,2.194356
682,id10,2024-09-23,163.620473,14.576458,67.0,0.089087,1.000000,1.155806,4.695720
683,id10,2024-09-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684,id10,2024-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### wLight

In [41]:
def preprocessing_wLight(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (sub, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': sub, 'date': date}
      for time_label, block_group in group.groupby('time_part'):
        light_list = block_group[col].tolist()

        # wLight 기초 통계
        mean = np.mean(light_list)
        row[f'{time_label}@wLight_mean'] = mean
        std = np.std(light_list)
        row[f'{time_label}@wLight_std'] = std
        cov = np.std(light_list) / np.mean(light_list)
        row[f'{time_label}@wLight_cov'] = cov
        max = np.max(light_list)
        # row[f'{time_label}@wLight_max'] = max
        min = np.min(light_list)
        # row[f'{time_label}@wLight_min'] = min
        range = max - min
        row[f'{time_label}@wLight_range'] = range

        # might 고유값
        light_cnt = len(set(light_list)) / len(light_list)
        row[f'{time_label}@wLight_cnt'] = light_cnt

        # wLight 다양성
        hist_light, _ = np.histogram(light_list, bins=10, density=True)
        ent = entropy(hist_light) if hist_light.sum() > 0 else 0
        row[f'{time_label}@wLight_entropy'] = ent

        # wLight 불안전성
        row[f'{time_label}@wLight_instability'] = (1 + ent) * (1 + cov) * (1 + light_cnt)

      results.append(row)

    return pd.DataFrame(results)

pre_wLight_df = preprocessing_wLight(wLight_df)

# def cummean_wLight(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_wLight_df = cummean_wLight(pre_wLight_df)

# def final_wLight(pre_wLight_df, cummean_wLight_df):
#   merge_df = pre_wLight_df.merge(cummean_wLight_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_wLight_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_wLight_df = final_wLight(pre_wLight_df, cummean_wLight_df)

In [42]:
pre_wLight_df

,subject_id,date,S3@wLight_mean,S3@wLight_std,S3@wLight_cov,S3@wLight_range,S3@wLight_cnt,S3@wLight_entropy,S3@wLight_instability
0,id01,2024-06-26,74.582609,98.368251,1.318917,264.0,0.191304,1.321190,6.412369
1,id01,2024-06-27,172.654762,131.143914,0.759573,293.0,0.142857,1.029495,4.081194
2,id01,2024-06-28,75.169231,126.372291,1.681170,311.0,0.092308,0.669392,4.889087
3,id01,2024-06-29,0.000000,0.000000,NaN,0.0,0.071429,0.000000,NaN
4,id01,2024-06-30,48.659091,111.873700,2.299133,311.0,0.068182,0.438159,5.068179
...,...,...,...,...,...,...,...,...,...
745,id10,2024-09-22,33.268908,37.009128,1.112424,184.0,0.168067,1.401174,5.924785
746,id10,2024-09-23,14.900000,11.126095,0.746718,23.0,0.066667,0.876939,3.497048
747,id10,2024-09-24,0.000000,0.000000,NaN,0.0,0.007246,0.000000,NaN
748,id10,2024-09-25,73.984962,2.011190,0.027184,11.0,0.022556,0.489382,1.564378


### wPedo

In [43]:
def preprocessing_wPedo(dfs):
  df = sleep_lifelog(dfs)
  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

  results = []

  for (subj, date), group in df.groupby(['subject_id', 'date']):
    row = {'subject_id':subj, 'date':date}
    for part, part_group in group.groupby('time_part'):
      step_list = []
      step_frequency_list = []
      distance_list = []
      speed_list = []
      burned_calories_list = []

      for _, entry in part_group.iterrows():
        step_list.append(entry['step'])
        step_frequency_list.append(entry['step_frequency'])
        distance_list.append(entry['distance'])
        speed_list.append(entry['speed'])
        burned_calories_list.append(entry['burned_calories'])

      # step 기초통계량
      step_sum = (np.sum(step_list) / len(step_list)) if sum(step_list) > 0 else 0
      row[f'{part}@step_sum'] = step_sum
      step_mean = np.mean(step_list) if sum(step_list) > 0 else 0
      row[f'{part}@step_mean'] = step_mean
      step_std = np.std(step_list) if sum(step_list) > 0 else 0
      row[f'{part}@step_std'] = step_std
      step_cov = (np.std(step_list) / np.mean(step_list)) if sum(step_list) > 0 else 0
      row[f'{part}@step_cov'] = step_cov

      # step 전이 수
      step_arr = np.array(step_list)
      transitions = np.sum(step_arr[1:] != step_arr[:-1]) / len(step_arr)
      row[f'{part}@step_transition'] = transitions

      # step 다양성
      hist, _ = np.histogram(step_arr, bins=10, density=True)
      step_entropy = entropy(hist) if hist.sum() > 0 else 0
      row[f'{part}@step_entropy'] = step_entropy

      # 보속 평균
      sf_mean = np.mean(step_frequency_list) if np.sum(step_frequency_list) > 0 else 0
      row[f'{part}@step_frequency_mean'] = sf_mean

      # 총 이동거리
      d_sum = np.sum(distance_list) / len(distance_list)
      row[f'{part}@distance_sum'] = d_sum

      # 속도 기초통계량
      speed_mean = np.mean(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_mean'] = speed_mean
      speed_std = np.std(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_std'] = speed_std
      speed_cov = (np.std(speed_list) / np.mean(speed_list)) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_cov'] = speed_std
      speed_max = np.max(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_max'] = speed_max

      # 총 소비된 칼로리
      bc_sum = (np.sum(burned_calories_list) / len(burned_calories_list)) if burned_calories_list and np.sum(burned_calories_list) > 0 else 0
      row[f'{part}@burned_calories_sum'] = bc_sum

      # 추정 보폭 평균
      if step_list and distance_list:
        step_sum = np.sum(step_list)
        if step_sum > 0:
          stride_mean = np.sum(distance_list) / step_sum
        else:
          stride_mean = 0
      else:
          stride_mean = 0
      row[f'{part}@stride_mean'] = stride_mean

      # pedo 불안전성
      instability = ((1 + step_entropy) * (1 + step_cov) *
       (1 + transitions) *
        (1 + speed_cov) * (1 + stride_mean))
      row[f'{part}@pedo_instability'] = instability

    results.append(row)

  return pd.DataFrame(results)

pre_wPedo_df = preprocessing_wPedo(wPedo_df)

# def cummean_wPedo(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_wPedo_df = cummean_wPedo(pre_wPedo_df)

# def final_wPedo(pre_wPedo_df, cummean_wPedo_df):
#   merge_df = pre_wPedo_df.merge(cummean_wPedo_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_wPedo_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_wPedo_df = final_wPedo(pre_wPedo_df, cummean_wPedo_df)

In [44]:
pre_wPedo_df

,subject_id,date,S3@step_sum,S3@step_mean,S3@step_std,S3@step_cov,S3@step_transition,S3@step_entropy,S3@step_frequency_mean,S3@distance_sum,S3@pedo_speed_mean,S3@pedo_speed_std,S3@pedo_speed_cov,S3@pedo_speed_max,S3@burned_calories_sum,S3@stride_mean,S3@pedo_instability
0,id01,2024-06-26,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,id01,2024-06-27,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,id01,2024-06-28,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,id01,2024-06-29,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,id01,2024-06-30,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,id10,2024-09-21,0.983333,0.983333,3.969992,4.03728,0.127778,0.366560,0.016389,0.701276,0.011688,0.047379,0.047379,0.431661,0.041167,0.713162,67.212468
730,id10,2024-09-22,0.420561,0.420561,2.867926,6.81929,0.056075,0.158655,0.007009,0.307480,0.005125,0.034900,0.034900,0.279842,0.024861,0.731120,129.363041
731,id10,2024-09-24,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
732,id10,2024-09-25,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


# 데이터 합치기

In [45]:
dfs_list = []
dfs = dataframe.copy()
for k, v in dfs.items():
  pre_name = f'pre_{k}_df'
  dfs_list.append(pre_name)
  print(pre_name)

# pre_list = {'pre_' + k + '_df2': v for k, v in dfs.items()}
# pre_name1

pre_mACStatus_df
pre_mActivity_df
pre_mAmbience_df
pre_mBle_df
pre_mGps_df
pre_mLight_df
pre_mScreenStatus_df
pre_mUsageStats_df
pre_mWifi_df
pre_wHr_df
pre_wLight_df
pre_wPedo_df


In [46]:
df_list = [pre_mACStatus_df,
pre_mActivity_df,
pre_mAmbience_df,
pre_mBle_df,
pre_mGps_df,
pre_mLight_df,
pre_mScreenStatus_df,
pre_mUsageStats_df,
pre_mWifi_df,
pre_wHr_df,
pre_wLight_df,
pre_wPedo_df]

In [47]:
prep_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list)

In [48]:
# 전처리된 데이터 저장
prep_df.to_csv('/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/prep_df_S3.csv',index=False)